In [1]:
# # Download the data (run it once!)
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet
# !wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv

In [2]:
import pandas as pd


# Read files
df_green_trip = pd.read_parquet("green_tripdata_2025-11.parquet")
print (df_green_trip.head())

print()

df_taxi_lookup = pd.read_csv("taxi_zone_lookup.csv")
print (df_taxi_lookup.head())


   VendorID lpep_pickup_datetime lpep_dropoff_datetime store_and_fwd_flag  \
0         2  2025-11-01 00:34:48   2025-11-01 00:41:39                  N   
1         2  2025-11-01 00:18:52   2025-11-01 00:24:27                  N   
2         2  2025-11-01 01:03:14   2025-11-01 01:15:24                  N   
3         2  2025-11-01 00:10:57   2025-11-01 00:24:53                  N   
4         1  2025-11-01 00:03:48   2025-11-01 00:19:38                  N   

   RatecodeID  PULocationID  DOLocationID  passenger_count  trip_distance  \
0         1.0            74            42              1.0           0.74   
1         1.0            74            42              2.0           0.95   
2         1.0            83           160              1.0           2.19   
3         1.0           166           127              1.0           5.44   
4         1.0           166           262              1.0           3.20   

   fare_amount  ...  mta_tax  tip_amount  tolls_amount  ehail_fee  \
0    

# Question 3. Counting short trips
For the trips in November 2025 (lpep_pickup_datetime between '2025-11-01' and '2025-12-01', exclusive of the upper bound), how many trips had a trip_distance of less than or equal to 1 mile?

In [3]:
# Convert pickup datetime to datetime type
df_green_trip["lpep_pickup_datetime"] = pd.to_datetime(df_green_trip["lpep_pickup_datetime"])

# Apply filters
df_q3 = df_green_trip[
    (df_green_trip["lpep_pickup_datetime"] >= "2025-11-01") &
    (df_green_trip["lpep_pickup_datetime"] < "2025-12-01") &
    (df_green_trip["trip_distance"] <= 1)
]

# Count trips
print(len(df_q3))

8007


# Question 4. Longest trip for each day
Which was the pick up day with the longest trip distance? Only consider trips with trip_distance less than 100 miles (to exclude data errors). Use the pick up time for your calculations.


In [4]:
df_q4 = df_green_trip.loc[
    df_green_trip["trip_distance"] < 100, 
    ["lpep_pickup_datetime", "trip_distance"]
].nlargest(1, "trip_distance")



In [5]:
# 1) Select only the columns needed
df_q4 = df_green_trip[["lpep_pickup_datetime", "trip_distance"]]

# 2) Apply the distance filter
df_q4 = df_q4[df_q4["trip_distance"] < 100]

# 3) Sort by distance descending
df_q4 = df_q4.sort_values("trip_distance", ascending=False)

# 4) Print the first row
df_q4.head(1)

# Alternative:
"""
df_q4 = df_green_trip.loc[
    df_green_trip["trip_distance"] < 100, 
    ["lpep_pickup_datetime", "trip_distance"]
].nlargest(1, "trip_distance")
"""


'\ndf_q4 = df_green_trip.loc[\n    df_green_trip["trip_distance"] < 100, \n    ["lpep_pickup_datetime", "trip_distance"]\n].nlargest(1, "trip_distance")\n'

# Question 5. Biggest pickup zone
Which was the pickup zone with the largest total_amount (sum of all trips) on November 18th, 2025?


In [6]:
# Filter by date
df_q5_trip = df_green_trip
df_q5_trip["lpep_pickup_datetime"] = pd.to_datetime(df_q5_trip["lpep_pickup_datetime"])
df_q5_trip = df_q5_trip[df_q5_trip["lpep_pickup_datetime"].dt.date == pd.to_datetime("2025-11-18").date()]

# Group by pickup location and sum totals
by_pu = df_q5_trip.groupby("PULocationID", as_index=False)["total_amount"].sum()

# Attach zone names
df_q5_location = df_taxi_lookup[["LocationID", "Zone"]]
by_pu = by_pu.merge(df_q5_location, left_on="PULocationID", right_on="LocationID", how="left")

# Top zone by total_amount
df_q5 = by_pu.nlargest(1, "total_amount")
print(df_q5[["PULocationID", "Zone", "total_amount"]])


    PULocationID               Zone  total_amount
39            74  East Harlem North       9281.92


# Question 6. Largest tip
For the passengers picked up in the zone named "East Harlem North" in November 2025, which was the drop off zone that had the largest tip?

Note: it's tip , not trip. We need the name of the zone, not the ID.


In [ ]:
# 1) Ensure pickup times are real datetimes
df_green_trip["lpep_pickup_datetime"] = pd.to_datetime(df_green_trip["lpep_pickup_datetime"])

# 2) Filter trips to November 2025
df_q6_trip = df_green_trip.loc[
    (df_green_trip["lpep_pickup_datetime"].dt.year == 2025)
    & (df_green_trip["lpep_pickup_datetime"].dt.month == 11),
    ["PULocationID", "DOLocationID", "tip_amount", "lpep_pickup_datetime"],
].copy()

# 3) Get LocationID for pickup zone "East Harlem North"
location_id = df_taxi_lookup.loc[df_taxi_lookup["Zone"] == "East Harlem North", "LocationID"].iloc[0]

# 4) Keep only trips picked up in that zone
df_q6_trip = df_q6_trip[df_q6_trip["PULocationID"] == location_id]

# 5) Attach dropoff zone names
drop_lookup = df_taxi_lookup[["LocationID", "Zone"]].rename(columns={"Zone": "dropoff_zone"})
df_q6_trip = df_q6_trip.merge(drop_lookup, left_on="DOLocationID", right_on="LocationID", how="left")

# 6) Show the dropoff zone with the largest tip
top_tip = df_q6_trip.loc[df_q6_trip["tip_amount"].idxmax(), ["dropoff_zone", "tip_amount"]]
print(top_tip)


dropoff_zone    Yorkville West
tip_amount               81.89
Name: 11613, dtype: object
